In [1]:
# preparing package, object, and library
import pandas as pd
import numpy as np 

In [2]:
# open the csv file 
mat_x_f1 =[]
mat_y_f1 =[]
mat_x_f2 =[]
mat_y_f2 =[]
mat_x_f3 =[]
mat_y_f3 =[]
# there are 35 dataset to test my model 
for i in range(35):
    
    pd_ind_feature1 = pd.read_csv("CKSNAP/data_cksnap_"+str(i)+ ".csv")
    pd_ind_feature2 = pd.read_csv("PseKNC/data_pseknc_"+str(i)+ ".csv")
    pd_ind_feature3 = pd.read_csv("PSTNPss/data_pstnpss_"+str(i)+ ".csv") # the feature that give the best accuracy
    
    # get the label every feature
    y_ind_feature1 = pd_ind_feature1["Target1"] # y_feature1 = y_feature2 = y_feature3 
    y_ind_feature2 = pd_ind_feature2["Target2"]
    y_ind_feature3 = pd_ind_feature3["Target3"]
    
    mat_y_f1.append(y_ind_feature1)
    mat_y_f2.append(y_ind_feature2)
    mat_y_f3.append(y_ind_feature3)
# delete column target
    del pd_ind_feature1["Target1"]
    del pd_ind_feature2["Target2"]
    del pd_ind_feature3["Target3"]

    x_feature1 = pd_ind_feature1
    x_feature2 = pd_ind_feature2
    x_feature3 = pd_ind_feature3
    
    mat_x_f1.append(x_feature1)
    mat_x_f2.append(x_feature2)
    mat_x_f3.append(x_feature3)


In [3]:
# combine the features 
#%run ./best_featureRNA.ipynb 
# merge the feature
# 1. CKSNAP(1) and PseKNC(2)
# 2. CKSNAP and PSTNPss(3)
# 3. PseKNC and PSTNPss
# 4. CKSNAP and PseKNC and PSTNPss
nCV = 10

# don't use numpy array
mat_x_merge1 =[]
mat_x_merge2 =[]
mat_x_merge3 =[]
mat_x_merge4 =[]

for i in range(35):
    X1 = mat_x_f1[i]
    X2 = mat_x_f2[i]
    X3 = mat_x_f3[i]

    X_merge1 = pd.concat([X1,X2], axis=1) # 1. CKSNAP and PseKNC
    X_merge2 = pd.concat([X1,X3], axis=1) # 2. CKSNAP and PSTNPss(3)
    X_merge3 = pd.concat([X2,X3], axis=1) # 3. PseKNC and PSTNPss
    X_merge4 = pd.concat([X1,X2,X3], axis=1) # 4. CKSNAP and PseKNC and PSTNPss
    mat_x_merge1.append(np.array(X_merge1,dtype="float64"))
    mat_x_merge2.append(np.array(X_merge2,dtype="float64"))
    mat_x_merge3.append(np.array(X_merge3,dtype="float64"))
    mat_x_merge4.append(np.array(X_merge4,dtype="float64"))

In [4]:
# load the list of bad feature
from numpy import load 
list_bad_feature1= load('list_bad_feature1.npy') 
list_bad_feature2= load('list_bad_feature2.npy') 
list_bad_feature3= load('list_bad_feature3.npy') 
list_bad_feature4= load('list_bad_feature4.npy') 

In [5]:
print(len(list_bad_feature1))
print(len(list_bad_feature2))
print(len(list_bad_feature3))
print(len(list_bad_feature4))

109
80
85
138


In [6]:
mat_reduced_f1=[]
mat_reduced_f2=[]
mat_reduced_f3=[]
mat_reduced_f4=[]

for i in range(35):
    mat_reduced_f1.append(np.delete(mat_x_merge1[i], list_bad_feature1, axis=1)) 
    mat_reduced_f2.append(np.delete(mat_x_merge2[i], list_bad_feature2, axis=1)) 
    mat_reduced_f3.append(np.delete(mat_x_merge3[i], list_bad_feature3, axis=1)) 
    mat_reduced_f4.append(np.delete(mat_x_merge4[i], list_bad_feature4, axis=1)) 

In [7]:
print(mat_reduced_f1[0].shape)
print(mat_reduced_f2[0].shape)
print(mat_reduced_f3[0].shape)
print(mat_reduced_f4[0].shape)


(1738, 21)
(1738, 23)
(1738, 20)
(1738, 31)


In [8]:
y= mat_y_f1[0]
y=np.array(y)
len(y)

1738

In [9]:
mat_reduced_f2[7]

array([[ 7.5000000e-02,  2.5000000e-02,  5.0000000e-02, ...,
        -4.6401130e-03,  4.5685230e-03,  1.0342149e-02],
       [ 2.5000000e-02,  0.0000000e+00,  0.0000000e+00, ...,
         6.8925560e-03, -5.7736260e-03,  1.0340823e-02],
       [ 1.5000000e-01,  1.0000000e-01,  7.5000000e-02, ...,
         9.2020070e-03,  2.2895640e-03, -1.9900000e-05],
       ...,
       [ 5.0000000e-02,  2.5000000e-02,  5.0000000e-02, ...,
         8.0737960e-03, -1.6106495e-02, -6.8832760e-03],
       [ 0.0000000e+00,  2.5000000e-02,  7.5000000e-02, ...,
        -1.1361650e-03,  3.1800000e-05,  1.1746110e-03],
       [ 5.0000000e-02,  2.5000000e-02,  2.5000000e-02, ...,
         1.1679830e-03,  1.1772630e-03,  1.1746110e-03]])

In [21]:
import joblib
loaded_model = joblib.load('Stacked_model/Stacked3_fold9.joblib')
for i in range(35):
    result = loaded_model.score(mat_reduced_f3[i], y)
    print(f'dataset number :{i} have score :{result}')

dataset number :0 have score :0.5281933256616801
dataset number :1 have score :0.45972382048331417
dataset number :2 have score :0.4096662830840046
dataset number :3 have score :0.46777905638665135
dataset number :4 have score :0.5828538550057537
dataset number :5 have score :0.501150747986191
dataset number :6 have score :0.520138089758343
dataset number :7 have score :0.5069044879171462
dataset number :8 have score :0.503452243958573
dataset number :9 have score :0.5454545454545454
dataset number :10 have score :0.503452243958573
dataset number :11 have score :0.5926352128883774
dataset number :12 have score :0.57307249712313
dataset number :13 have score :0.5161104718066744
dataset number :14 have score :0.5028768699654775
dataset number :15 have score :0.5840046029919448
dataset number :16 have score :0.5477560414269275
dataset number :17 have score :0.5327963176064442
dataset number :18 have score :0.5069044879171462
dataset number :19 have score :0.5120828538550057
dataset number

In [29]:
# based on experiments, dataset number 15 is the best in accuracy score
# check the score in every fold 
total_y_pred_test =[]
total_y_true_test=[]
for i in range(10): # i use 10 fold so, i have 10 models
    loaded_model = joblib.load('Stacked_model/Stacked3_fold'+str(i)+'.joblib')
    result = loaded_model.score(mat_reduced_f3[15], y)
    y_pred_test = loaded_model.predict(mat_reduced_f3[15])
    total_y_pred_test.append(y_pred_test)
    total_y_true_test.append(y)
  
    print(f' fold number {i} scores {result}')

 fold number 0 scores 0.6294591484464902
 fold number 1 scores 0.6052934407364787
 fold number 2 scores 0.5828538550057537
 fold number 3 scores 0.5736478711162255
 fold number 4 scores 0.6237054085155351
 fold number 5 scores 0.6012658227848101
 fold number 6 scores 0.6121979286536249
 fold number 7 scores 0.6283084004602992
 fold number 8 scores 0.6352128883774454
 fold number 9 scores 0.5840046029919448


In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
def Total_performances_ind_dataset(k,total_y_pred_test,total_y_true_test):
    total_cm_test=[]
    total_accuracy_test=[]  
    total_sensitivity_test=[]
    total_specificity_test=[]
    total_MCC_test =[]
    total_AUC_test =[]

    for j in range(k):
       
        y_pred_test = total_y_pred_test[j] 
        y_true_test = total_y_true_test[j]
    
        for i in range(len(y_pred_test)):
            if y_pred_test[i] <0.5:
                y_pred_test[i] = 0
            else :
                y_pred_test[i] = 1   
            
        total_y_pred_test.append(y_pred_test)
        total_cm_test.append(confusion_matrix(y_true_test, y_pred_test))
    
    
        
        total_MCC_test.append(matthews_corrcoef(y_true_test, y_pred_test))     
        
        fpr2, tpr2, thresholds2 = roc_curve(y_true_test, y_pred_test, pos_label=1)
       
        total_AUC_test.append(auc(fpr2,tpr2))
        
    for i in range(k):
       
        c_m_test = total_cm_test[i]
       
        total_cmtest=sum(sum(c_m_test))
    
       
        testAccuracy = (c_m_test[0,0]+c_m_test[1,1])/total_cmtest
    
       
        testSensitivity = c_m_test[1,1]/(c_m_test[1,0]+c_m_test[1,1])
    
       
        testSpecificity = c_m_test[0,0]/(c_m_test[0,0]+c_m_test[0,1])               
    
       
        total_accuracy_test.append(testAccuracy)
        
        total_sensitivity_test.append(testSensitivity)                      
        
        total_specificity_test.append(testSpecificity)
        
    for i in range (k): 
        
        total_accuracy_test[i]     =  round(total_accuracy_test[i],4)
       
        total_sensitivity_test[i]  =  round(total_sensitivity_test[i],4)
        
        total_specificity_test[i]  =  round(total_specificity_test[i],4)
        
        total_MCC_test[i]          =  round(total_MCC_test[i],4)
       
        total_AUC_test[i]          =  round(total_AUC_test[i],4)     
        
        total_performances_test = [total_accuracy_test,total_sensitivity_test,total_specificity_test, total_MCC_test,total_AUC_test]
    return total_performances_test

In [32]:
total_performances_test = Total_performances_ind_dataset(10,total_y_pred_test,total_y_true_test)

In [35]:
import statistics
acc_mean = statistics.mean(total_performances_test[0])
sn_mean =statistics.mean(total_performances_test[1])
sp_mean =statistics.mean(total_performances_test[2])
mcc_mean =statistics.mean(total_performances_test[3])
auc_mean=statistics.mean(total_performances_test[4])

print(f'acc : {acc_mean}')
print(f'sn : {sn_mean}')
print(f'sp : {sp_mean}')
print(f'mcc : {mcc_mean}')
print(f'auc : {auc_mean}')


acc : 0.6075999999999999
sn : 0.63568
sp : 0.57953
mcc : 0.21589
auc : 0.6075999999999999
